In [1]:
from dsm import datasets, DeepRecurrentSurvivalMachines

In [2]:
import numpy as np

In [3]:
import pkgutil


In [4]:
def unrollx(data):
     return np.vstack([dat for dat in data])
def unrollt(data):
     return np.concatenate([dat for dat in data])

In [5]:
x, t, e = datasets._load_framingham_dataset( sequential=True)

In [6]:
x, t, e = np.array(x), np.array(t), np.array(e)

In [7]:
# from matplotlib import pyplot as plt

# plt.hist(unrollt(t)[unrollt(e)==1], bins=25)

In [8]:
times = list(np.quantile(unrollt(t)[unrollt(e)==1], [0.25, .5, 0.75]))

In [9]:
times

[2254.5, 3913.0, 5766.0]

In [22]:
folds = np.array((list(range(5))*10000)[:len(x)])

In [23]:
unrollt(t)


array([8766, 4138, 8766, ..., 8766, 6376, 4565], dtype=int64)

In [24]:
from lifelines import CoxPHFitter

In [25]:
import pandas as pd

In [26]:
def convert_to_data_frame(x, t, e):

  df = pd.DataFrame(data=x, columns=['X' + str(i) for i in range(x.shape[1])])
  df['T'] = pd.DataFrame(data=t.reshape(-1, 1), columns=['T'])
  df['E'] = pd.DataFrame(data=e.reshape(-1, 1), columns=['E'])

  return df

In [27]:
from sksurv.metrics import concordance_index_ipcw, brier_score

In [28]:
cis = []
brs = []

for fold in set(folds):  
    
    x_tr, t_tr, e_tr = x[folds!=fold], t[folds!=fold], e[folds!=fold]
    x_te, t_te, e_te = x[folds==fold], t[folds==fold], e[folds==fold]
    
    x_tr = unrollx(x_tr)
    t_tr = unrollt(t_tr)
    e_tr = unrollt(e_tr)

    x_te = unrollx(x_te)
    t_te = unrollt(t_te)
    e_te = unrollt(e_te)
    
    df_tr = convert_to_data_frame(x_tr, t_tr, e_tr)

    model = CoxPHFitter(penalizer=1e-3).fit(df_tr, duration_col='T', event_col='E')
    
    preds = model.predict_survival_function(x_te, times).T.values
    
    et_tr = np.array([(e_tr[i], t_tr[i]) for i in range(len(e_tr))],
                 dtype=[('e', bool), ('t', int)])
    et_te = np.array([(e_te[i], t_te[i]) for i in range(len(e_te))],
                 dtype=[('e', bool), ('t', int)])
    
    print (preds.shape)
    
    cis_ = []
    for i in range(len(times)):
        cis_.append(concordance_index_ipcw(et_tr, et_te, 1-preds[:,i], times[i])[0])
    cis.append(cis_)
    
    brs.append(brier_score(et_tr, et_te, preds, times )[1])

print (np.mean(cis, axis=0))
print (np.mean(brs, axis=0))

(2329, 3)
(2314, 3)
(2314, 3)
(2343, 3)
(2327, 3)
[0.7692282  0.76340607 0.763032  ]
[0.06235129 0.10534096 0.13164272]


In [29]:
from dsm import DeepSurvivalMachines

In [30]:
cis = []
brs = []

for fold in set(folds):  
    
    x_tr, t_tr, e_tr = x[folds!=fold], t[folds!=fold], e[folds!=fold]
    x_te, t_te, e_te = x[folds==fold], t[folds==fold], e[folds==fold]
    
    x_tr = unrollx(x_tr)
    t_tr = unrollt(t_tr)
    e_tr = unrollt(e_tr)

    x_te = unrollx(x_te)
    t_te = unrollt(t_te)
    e_te = unrollt(e_te)
    
    df_tr = convert_to_data_frame(x_tr, t_tr, e_tr)

    model = DeepSurvivalMachines(k=6, layers=[100])
    model.fit(x_tr, t_tr, e_tr, iters=100, batch_size=20, learning_rate=1e-3)
    
    #print (preds)
    
    preds = model.predict_survival(x_te, times)
    
    #print (preds.shape)

    
    et_tr = np.array([(e_tr[i], t_tr[i]) for i in range(len(e_tr))],
                 dtype=[('e', bool), ('t', int)])
    et_te = np.array([(e_te[i], t_te[i]) for i in range(len(e_te))],
                 dtype=[('e', bool), ('t', int)])
    
    cis_ = []
    for i in range(len(times)):
        cis_.append(concordance_index_ipcw(et_tr, et_te, 1-preds[:,i], times[i])[0])
    cis.append(cis_)
    
    brs.append(brier_score(et_tr, et_te, preds, times )[1])

print (np.mean(cis, axis=0))
print (np.mean(brs, axis=0))

  7%|▋         | 7/100 [00:16<03:42,  2.39s/it]


[0.77129777 0.76375409 0.76286037]
[0.06251423 0.10639523 0.13264601]


In [54]:
unrollx(x).shape

(11627, 18)

In [57]:
preds_.shape

(2329, 3)

In [51]:
preds

array([1., 1., 1., ..., 1., 1., 1.])

In [46]:
[0.77528508 0.76668518 0.76477279]
[0.06211699 0.10525421 0.13180519]


SyntaxError: invalid syntax (<ipython-input-46-c6b233be65e1>, line 1)

In [61]:
from dsm import DeepRecurrentSurvivalMachines

In [62]:
cis = []
brs = []

for fold in set(folds):  
    
    x_tr, t_tr, e_tr = x[folds!=fold], t[folds!=fold], e[folds!=fold]
    x_te, t_te, e_te = x[folds==fold], t[folds==fold], e[folds==fold]
    

#     df_tr = convert_to_data_frame(x_tr, t_tr, e_tr)

    model = DeepRecurrentSurvivalMachines(k=6, layers=1, hidden=50,discount=1, typ='RNN', temp=100)
    model.fit(x_tr, t_tr, e_tr, iters=100, batch_size=10, learning_rate=1e-3)
    
    #print (preds)
    
    preds = model.predict_survival(x_te, times)
    
    x_tr = unrollx(x_tr)
    t_tr = unrollt(t_tr)
    e_tr = unrollt(e_tr)

    x_te = unrollx(x_te)
    t_te = unrollt(t_te)
    e_te = unrollt(e_te)
    
    et_tr = np.array([(e_tr[i], t_tr[i]) for i in range(len(e_tr))],
                 dtype=[('e', bool), ('t', int)])
    et_te = np.array([(e_te[i], t_te[i]) for i in range(len(e_te))],
                 dtype=[('e', bool), ('t', int)])
    
    cis_ = []
    for i in range(len(times)):
        cis_.append(concordance_index_ipcw(et_tr, et_te, 1-preds[:,i], times[i])[0])
    cis.append(cis_)
    
    brs.append(brier_score(et_tr, et_te, preds, times )[1])

print (np.mean(cis, axis=0))
print (np.mean(brs, axis=0))


  1%|          | 55/10000 [00:00<00:18, 545.25it/s]

(3547, 3, 18)


  0%|          | 36/10000 [00:00<00:28, 355.03it/s]

(3547, 3, 18)


  0%|          | 43/10000 [00:00<00:23, 424.65it/s]

...

(3547, 3, 18)


  1%|          | 52/10000 [00:00<00:19, 517.07it/s]

(3548, 3, 18)


  6%|▌         | 6/100 [00:07<02:01,  1.30s/it]

[0.77460045 0.7664278  0.76401525]
[0.06221371 0.10567765 0.13565704]


In [20]:
model = DeepRecurrentSurvivalMachines(layers=1, hidden=10)


In [24]:
model.layers


In [ ]:
cis = []
brs = []

for fold in set(folds):  
    
    x_tr, t_tr, e_tr = x[folds!=fold], t[folds!=fold], e[folds!=fold]
    x_te, t_te, e_te = x[folds==fold], t[folds==fold], e[folds==fold]
    
    x_tr = unrollx(x_tr)
    t_tr = unrollt(t_tr)
    e_tr = unrollt(e_tr)

    x_te = unrollx(x_te)
    t_te = unrollt(t_te)
    e_te = unrollt(e_te)
    
    df_tr = convert_to_data_frame(x_tr, t_tr, e_tr)
    
        
    et_tr = np.array([(e_tr[i], t_tr[i]) for i in range(len(e_tr))],
                 dtype=[('e', bool), ('t', int)])
    et_te = np.array([(e_te[i], t_te[i]) for i in range(len(e_te))],
                 dtype=[('e', bool), ('t', int)])
    
    model = RandomSurvivalForestModel(num_trees=50)
    model = model.fit(x_tr, t_tr, e_tr)
    
    preds = []
    for time in times:
        preds.append(model.predict_survival(x_te, time))
    print (len(preds))
    
    cis_ = []
    for i in range(len(times)):
        cis_.append(concordance_index_ipcw(et_tr, et_te, 1-preds[:,i], times[i])[0])
    cis.append(cis_)
    
    brs.append(brier_score(et_tr, et_te, preds, times )[1])


In [ ]:
model.predict_survival_function(x_te, times).T.values

In [91]:
[risk.y[3] for risk in out_risk]

[0.3126132634036184,
 0.46647488732207254,
 0.5624185126209059,
 0.4059503413491215,
 0.4415347567963127,
 0.2670742533214408,
 0.06359725383083349,
 0.9349522484067815,
 0.9486941636172256,
 0.9522407447171799,
 0.9103188179868816,
 0.8824124615716761,
 0.7801756137064368,
 0.752123603797489,
 0.5288001066166764,
 0.7803207243582894,
 0.46932603202272666,
 0.9289505993097185,
 0.9355269013394406,
 0.9575783226069599,
 0.9381858563916292,
 0.9595487184435972,
 0.9310981298806621,
 0.9326698824044939,
 0.9011583356828196,
 0.9229446679118422,
 0.8835080122599095,
 0.900997315998892,
 0.8925830090019774,
 0.8831209397700821,
 0.8005351813156036,
 0.7216661520824456,
 0.7599841537178933,
 0.00015928860304218303,
 0.7890039749760369,
 0.9350205039206175,
 0.9072527188714308,
 0.9755350533464282,
 0.9857568595759039,
 0.9678514283741088,
 0.9640084861264626,
 0.7937728360920354,
 0.7358192205521833,
 0.8280615261984754,
 0.6155861711521265,
 0.765769361328416,
 0.06586886769950233,
 0.16902